# Using Hand Crafted Features

In [1]:
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score

In [2]:
# Paths to dataset
dataset_path = "classification_dataset"  # Using raw strings bacause the file names contain backslashes and escape characters
mask_path = os.path.join(dataset_path, "with_mask")
no_mask_path = os.path.join(dataset_path, "without_mask")

In [3]:
#loading a sample image to show that images with 0_0_≈˙◊¢ are not loading correctly
sample_image = cv2.imread(r'classification_dataset\with_mask\0_0_≈˙◊¢ 2020-02-23 132115.png')
if sample_image is None:
    print('Image not loaded correctly')


Image not loaded correctly


[ WARN:0@1.310] global loadsave.cpp:268 findDecoder imread_('classification_dataset\with_mask\0_0_≈˙◊¢ 2020-02-23 132115.png'): can't open/read file: check file path/integrity


This shows that the images with this kind of a name are not being loaded.Manual inspection shows that these kind of images only exist in the with_mask category.

In [4]:
# Now we create a python script to rename all the wrongly names images
import os

# Define the folder path where images are stored
folder_path = mask_path

# Define the characters to be replaced
special_chars = "≈˙◊¢"

# List all files in the folder
for filename in os.listdir(folder_path):
    old_path = os.path.join(folder_path, filename)

    # Replace specific special characters with '_'
    new_filename = filename
    for char in special_chars:
        new_filename = new_filename.replace(char, "_")

    new_path = os.path.join(folder_path, new_filename)

    # Rename the file if necessary
    if old_path != new_path:
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} → {new_filename}")

print("Renaming completed!")



Renaming completed!


Now there will be no problem in loading any of the images.

In [5]:
# Feature extraction functions
def extract_hog_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features, _ = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), 
                      block_norm='L2-Hys', visualize=True)
    return features

def extract_lbp_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, P=8, R=1, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 10))
    hist = hist.astype("float")
    hist /= hist.sum()
    return hist

def extract_color_histogram(image, bins=(8, 8, 8)):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

In [6]:
# Iterate directory
def count_images(dir_path):
    count = 0
    for path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, path)):
            count += 1
    return count
print('Images with Mask:',count_images(mask_path))
print('Images without Mask:',count_images(no_mask_path))


Images with Mask: 2165
Images without Mask: 1930


In [7]:
# Load dataset and extract features
X, y = [], []

for label, path in enumerate([mask_path, no_mask_path]):  # 0: with_mask, 1: without_mask
    for file in os.listdir(path):
        img_path = os.path.join(path, file)  # Use raw string path
        image = cv2.imread(img_path)
        if image is not None:
            image = cv2.resize(image, (128, 128))
            # Extract features
            hog_feat = extract_hog_features(image)
            lbp_feat = extract_lbp_features(image)
            color_feat = extract_color_histogram(image)
            # Combine features
            combined_features = np.hstack([hog_feat, lbp_feat, color_feat])
            X.append(combined_features)
            y.append(label)
        else:
            print(f"Error loading image: {img_path}")

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [8]:
# **Simplified Train-Validation-Test Split (70%-15%-15%)**
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [9]:
# Train SVC
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X_train, y_train)
svm_val_preds = svm.predict(X_val)
svm_f1 = f1_score(y_val, svm_val_preds)
print(classification_report(y_val, svm_val_preds))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90       332
           1       0.88      0.89      0.88       282

    accuracy                           0.89       614
   macro avg       0.89      0.89      0.89       614
weighted avg       0.89      0.89      0.89       614



In [10]:
# Train Neural Network Classifier
nn = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
nn.fit(X_train, y_train)
nn_val_preds = nn.predict(X_val)
nn_f1 = f1_score(y_val, nn_val_preds)
print(classification_report(y_val, nn_val_preds))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93       332
           1       0.92      0.93      0.92       282

    accuracy                           0.93       614
   macro avg       0.93      0.93      0.93       614
weighted avg       0.93      0.93      0.93       614



In [11]:
# Train XGBoost Classifier
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42,n_jobs=-1)
xgb.fit(X_train, y_train)
xgb_val_preds = xgb.predict(X_val)
xgb_f1 = f1_score(y_val, xgb_val_preds)
print(classification_report(y_val, xgb_val_preds))

              precision    recall  f1-score   support

           0       0.91      0.96      0.93       332
           1       0.95      0.88      0.91       282

    accuracy                           0.92       614
   macro avg       0.93      0.92      0.92       614
weighted avg       0.92      0.92      0.92       614



In [12]:
# **Choose Best Model Based on F1-Score**
best_model_name, best_model, best_f1_score = max(
    zip(["SVM", "Neural Network", "XGBoost"], 
        [svm, nn, xgb], 
        [svm_f1, nn_f1, xgb_f1]), 
    key=lambda x: x[2]
)

# **Final Testing on the Best Model**
final_test_preds = best_model.predict(X_test)
final_test_f1 = f1_score(y_test, final_test_preds)
final_test_report = classification_report(y_test, final_test_preds)

# **Print Results**
print(f"Validation F1-Scores: SVM={svm_f1:.4f}, NN={nn_f1:.4f}, XGBoost={xgb_f1:.4f}")
print(f"Best Model: {best_model_name} with Validation F1-Score = {best_f1_score:.4f}")
print(f"Test F1-Score for Best Model ({best_model_name}): {final_test_f1:.4f}")
print(final_test_report)

Validation F1-Scores: SVM=0.8834, NN=0.9206, XGBoost=0.9138
Best Model: Neural Network with Validation F1-Score = 0.9206
Test F1-Score for Best Model (Neural Network): 0.9372
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       340
           1       0.93      0.95      0.94       275

    accuracy                           0.94       615
   macro avg       0.94      0.94      0.94       615
weighted avg       0.94      0.94      0.94       615



# Using CNN(Automatic Feature Learning)

In [13]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Rescaling, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.metrics import f1_score, accuracy_score

2025-03-25 12:39:10.932968: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 12:39:11.200481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742886551.297116    2951 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742886551.322134    2951 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-25 12:39:11.549080: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [14]:
# def rename(dataset_path,string):
#     # Get all image files
#     image_files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]

#     # Rename files sequentially
#     for index, filename in enumerate(image_files, start=1):
#         old_path = os.path.join(dataset_path, filename)

#         # Extract the file extension (e.g., .jpg, .png)
#         extension = os.path.splitext(filename)[1]  # Includes the dot

#         # Generate new filename
#         new_filename = f"image_{string}_{index}{extension}"
#         new_path = os.path.join(dataset_path, new_filename)

#         # Rename the file
#         os.rename(old_path, new_path)
#         # print(f"Renamed: {filename} → {new_filename}")

# print(" All images renamed")

# rename(r"classification_dataset/with_mask", 'with_mask')
# rename(r"classification_dataset/without_mask", 'without_mask')



In [15]:
# Load datasets
dataset_path = "classification_dataset"  # Using raw strings bacause the file names contain backslashes and escape characters

batch_size = 32
img_size = (128, 128)
train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.3,  
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

Found 4092 files belonging to 2 classes.
Using 2865 files for training.


I0000 00:00:1742886553.459515    2951 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6196 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [16]:

val_test_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.3,  
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

Found 4092 files belonging to 2 classes.
Using 1227 files for validation.


In [17]:
# **Further split val_test into validation (15%) and test (15%)**
val_size = int(0.5 * len(val_test_ds))  # 50% of remaining 30% goes to validation
val_ds = val_test_ds.take(val_size)
test_ds = val_test_ds.skip(val_size)

In [18]:
# # **Data Augmentation (Mapped to Training Set)**
# def augment(image, label):
#     image = tf.image.random_flip_left_right(image)
#     image = tf.image.random_rotation(image, 0.1)
#     return image, label

# train_ds = train_ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)

In [19]:
# **Enable Caching & Prefetching for Faster Performance**
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Rescaling
from tensorflow.keras.optimizers import Adam

def create_cnn(activation='relu', optimizer=Adam(learning_rate=0.001), dropout_rate=0.5):
    model = Sequential([
        Input(shape=(128, 128, 3)),  # Explicit Input Layer
        Rescaling(1./255),  # Rescale pixel values
        
        Conv2D(32, (3,3), activation=activation, padding='valid'),
        BatchNormalization(),
        MaxPooling2D((2,2)),

        Conv2D(64, (3,3), activation=activation, padding='valid'),
        BatchNormalization(),
        MaxPooling2D((2,2)),

        Flatten(),
        Dense(128, activation=activation),
        Dropout(dropout_rate),  # Dropout to prevent overfitting
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [21]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Define early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,         # Stop after 10 epochs with no improvement
    restore_best_weights=True,  # Restore best weights
    verbose=1
)

# Create model
model = create_cnn(activation='relu', optimizer=Adam(0.0001), dropout_rate=0.5)

# Train model with early stopping
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    batch_size=16,
    verbose=1,
    callbacks=[early_stopping]  # Add early stopping
)


Epoch 1/50


2025-03-25 12:39:15.675244: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
2025-03-25 12:39:15.938297: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
I0000 00:00:1742886556.163950    4345 service.cc:148] XLA service 0x7d722000db70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742886556.166923    4345 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-03-25 12:39:16.207305: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742886556.371324    4345 cuda_dnn.cc:529] Loaded cuDNN version 90300


13/90 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7280 - loss: 1.0049

I0000 00:00:1742886558.299110    4345 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.8424 - loss: 0.6568 - val_accuracy: 0.4655 - val_loss: 1.4383
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9568 - loss: 0.1135 - val_accuracy: 0.4704 - val_loss: 3.0626
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9863 - loss: 0.0509 - val_accuracy: 0.5247 - val_loss: 3.2846
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9806 - loss: 0.0482 - val_accuracy: 0.7780 - val_loss: 1.1368
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9901 - loss: 0.0258 - val_accuracy: 0.8355 - val_loss: 0.8419
Epoch 6/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9922 - loss: 0.0226 - val_accuracy: 0.9178 - val_loss: 0.3918
Epoch 7/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9880 - loss: 0.0321 - val_accuracy: 0.9309 - val_loss: 0.4594
Epoch 8/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9912 - loss: 0.0293 - val_accuracy: 0.9507 - val_loss: 0.

In [22]:
# **Hyperparameter Variations**
learning_rates = [0.001, 0.0001]
batch_sizes = [16, 32]
optimizers = [Adam, RMSprop]
activations = ['relu','elu']
dropout_rates = [0.3, 0.5]

# **Training and Comparing Different CNNs**
results = []

for lr in learning_rates:
    for batch_size in batch_sizes:
        for optimizer in optimizers:
            for activation in activations:
                for dropout_rate in dropout_rates:
                    print(f"\nTraining CNN with: LR={lr}, Batch={batch_size}, Optimizer={optimizer.__name__}, Activation={activation}, Dropout={dropout_rate}")

                    # Create and train CNN
                    # Define early stopping
                    early_stopping = EarlyStopping(
                                                    monitor='val_loss',  # Monitor validation loss
                                                    patience=5,         # Stop after 10 epochs with no improvement
                                                    restore_best_weights=True,  # Restore best weights
                                                    verbose=0
                                                )
                    model = create_cnn(activation=activation, optimizer=optimizer(learning_rate=lr), dropout_rate=dropout_rate)
                    history = model.fit(
                                        train_ds,
                                        validation_data=val_ds,
                                        epochs=50,
                                        batch_size=batch_size,
                                        verbose=0,
                                        callbacks=[early_stopping]  # Add early stopping
                                        )
                    # Evaluate on test set
                    y_true = []
                    y_pred = []
                    for images, labels in test_ds:
                        preds = model.predict(images,verbose=0) > 0.5
                        y_true.extend(labels.numpy())
                        y_pred.extend(preds.astype("int32").flatten())

                    f1 = f1_score(y_true, y_pred)
                    acc = accuracy_score(y_true, y_pred)

                    # Store results
                    results.append({
                        'Learning Rate': lr,
                        'Batch Size': batch_size,
                        'Optimizer': optimizer.__name__,
                        'Activation': activation,
                        'Dropout Rate': dropout_rate,
                        'Test Accuracy': acc,
                        'Test F1-Score': f1
                    })

# **Find the Best Model Based on F1-Score**
best_model = max(results, key=lambda x: x['Test F1-Score'])



Training CNN with: LR=0.001, Batch=16, Optimizer=Adam, Activation=relu, Dropout=0.3


2025-03-25 12:39:57.352343: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence



Training CNN with: LR=0.001, Batch=16, Optimizer=Adam, Activation=relu, Dropout=0.5


2025-03-25 12:40:22.647526: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence



Training CNN with: LR=0.001, Batch=16, Optimizer=Adam, Activation=elu, Dropout=0.3

Training CNN with: LR=0.001, Batch=16, Optimizer=Adam, Activation=elu, Dropout=0.5


...

2025-03-25 12:42:16.862191: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence



Training CNN with: LR=0.001, Batch=32, Optimizer=Adam, Activation=relu, Dropout=0.3

Training CNN with: LR=0.001, Batch=32, Optimizer=Adam, Activation=relu, Dropout=0.5

Training CNN with: LR=0.001, Batch=32, Optimizer=Adam, Activation=elu, Dropout=0.3

Training CNN with: LR=0.001, Batch=32, Optimizer=Adam, Activation=elu, Dropout=0.5

Training CNN with: LR=0.001, Batch=32, Optimizer=RMSprop, Activation=relu, Dropout=0.3

Training CNN with: LR=0.001, Batch=32, Optimizer=RMSprop, Activation=relu, Dropout=0.5

Training CNN with: LR=0.001, Batch=32, Optimizer=RMSprop, Activation=elu, Dropout=0.3

Training CNN with: LR=0.001, Batch=32, Optimizer=RMSprop, Activation=elu, Dropout=0.5


2025-03-25 12:45:08.774687: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence



Training CNN with: LR=0.0001, Batch=16, Optimizer=Adam, Activation=relu, Dropout=0.3

Training CNN with: LR=0.0001, Batch=16, Optimizer=Adam, Activation=relu, Dropout=0.5

Training CNN with: LR=0.0001, Batch=16, Optimizer=Adam, Activation=elu, Dropout=0.3

Training CNN with: LR=0.0001, Batch=16, Optimizer=Adam, Activation=elu, Dropout=0.5

Training CNN with: LR=0.0001, Batch=16, Optimizer=RMSprop, Activation=relu, Dropout=0.3

Training CNN with: LR=0.0001, Batch=16, Optimizer=RMSprop, Activation=relu, Dropout=0.5

Training CNN with: LR=0.0001, Batch=16, Optimizer=RMSprop, Activation=elu, Dropout=0.3

Training CNN with: LR=0.0001, Batch=16, Optimizer=RMSprop, Activation=elu, Dropout=0.5

Training CNN with: LR=0.0001, Batch=32, Optimizer=Adam, Activation=relu, Dropout=0.3

Training CNN with: LR=0.0001, Batch=32, Optimizer=Adam, Activation=relu, Dropout=0.5

Training CNN with: LR=0.0001, Batch=32, Optimizer=Adam, Activation=elu, Dropout=0.3

Training CNN with: LR=0.0001, Batch=32, Optimi

2025-03-25 12:50:24.185293: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [23]:

# Print Best Model Configuration and Results
print("\n Best CNN Configuration:")
print(f"Learning Rate: {best_model['Learning Rate']}")
print(f"Batch Size: {best_model['Batch Size']}")
print(f"Optimizer: {best_model['Optimizer']}")
print(f"Activation: {best_model['Activation']}")
print(f"Test Accuracy: {best_model['Test Accuracy']:.4f}")
print(f"Test F1-Score: {best_model['Test F1-Score']:.4f}")


 Best CNN Configuration:
Learning Rate: 0.0001
Batch Size: 16
Optimizer: RMSprop
Activation: relu
Test Accuracy: 0.9677
Test F1-Score: 0.9672


# Comparing the results

In [24]:
import pandas as pd

# Extract the best CNN results from the hyperparameter tuning process
best_cnn_results = {
    "Model": "Best CNN",
    "Test Accuracy": best_model["Test Accuracy"],  # Best CNN Accuracy
    "Test F1-Score": best_model["Test F1-Score"]  # Best CNN F1-Score
}

# Extract handcrafted feature-based models' best results
handcrafted_results = [
    {"Model": "SVM", "Test Accuracy": accuracy_score(y_test, svm.predict(X_test)), "Test F1-Score": f1_score(y_test, svm.predict(X_test))},
    {"Model": "XGBoost", "Test Accuracy": accuracy_score(y_test, xgb.predict(X_test)), "Test F1-Score": f1_score(y_test, xgb.predict(X_test))},
    {"Model": "Neural Network (MLP)", "Test Accuracy": accuracy_score(y_test, nn.predict(X_test)), "Test F1-Score": f1_score(y_test, nn.predict(X_test))}
]

# Convert to DataFrame
comparison_results = pd.DataFrame(handcrafted_results + [best_cnn_results])
print(comparison_results.head(100))


                  Model  Test Accuracy  Test F1-Score
0                   SVM       0.933333       0.927690
1               XGBoost       0.951220       0.944853
2  Neural Network (MLP)       0.943089       0.937163
3              Best CNN       0.967690       0.967213
